In [46]:
import pandas as pd
import re
import collections
import pprint

In [5]:
data = pd.read_csv('reg-condition.csv', delimiter = ',')

data.head()

,courseId,courseNumber,condition
0,25468,100316,100112 and third year standing; or 100112 and ...
1,6760,206787,"206332 and one year courses in statistic, basi..."
2,22660,703798,Students are required to submit independent st...
3,92,1320,001101 AND 001102 AND 001201 AND 001202; OR 00...
4,24731,204482,204203 or 204211 or 204214 or 204215 or 204216...


In [50]:
token = []
head = data.head(10)

for index, row in data.iterrows() :
	condition = str(row[2]).lower()
	splited = re.split(';| and | or ', condition)
	splited = [(lambda s : s.strip())(s) for s in splited]
	splited = list(filter(None, splited))
	for s in splited :
		token.append(s)

counter = collections.Counter(token)

with open('output.txt', 'w') as output :
	pprint.pprint(dict(counter), output)

- one year courses in statistic, basic concepts of set theory
- some probabilistic concept

- ; is 'and' or 'or'